# 3.2 Calling differentially expressed peaks

## Missing R packages 

When running the scripts in this section, if you get an error saying the **locfit** package has not been installed, you have two options: (1) either run the following command to add a pre-downloaded version of the package to the **$R_LIBS** environment variable...

In [12]:
%%bash
export R_LIBS=$R_LIBS:/users/avanti/R/x86_64-unknown-linux-gnu-library/3.0/


...or install the package locally by executing the following cell. The code in this cell will start the R shell, and then run install.packages("locfit").

In [7]:
%%bash
R --no-save 
install.packages("locfit",repos='http://cran.us.r-project.org')
q() 


R version 3.2.3 (2015-12-10) -- "Wooden Christmas-Tree"
Copyright (C) 2015 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages("locfit",repos='http://cran.us.r-project.org')
gcc -std=gnu99 -I/usr/share/R/include -DNDEBUG      -fpic  -g -O2 -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security -D_FORTIFY_SOURCE=2 -g  -c S_enter.c -o S_enter.o
gcc -std=gnu99 -I/usr/share/R/include -DNDEB

Installing package into ‘/users/annashch/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/locfit_1.5-9.1.tar.gz'
Content type 'application/x-gzip' length 196560 bytes (191 KB)
downloaded 191 KB

* installing *source* package ‘locfit’ ...
** package ‘locfit’ successfully unpacked and MD5 sums checked
** libs
installing to /users/annashch/R/x86_64-pc-linux-gnu-library/3.2/locfit/libs
** R
** data
** preparing package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded
* DONE (locfit)


In additional, you will need DESeq installed. You can do this by using the **export R_LIBS...** command above, or by executing the following cell of R code: 

In [8]:
%%bash
R --no-save
source("http://bioconductor.org/biocLite.R")
biocLite("DESeq")
q() 


R version 3.2.3 (2015-12-10) -- "Wooden Christmas-Tree"
Copyright (C) 2015 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> source("http://bioconductor.org/biocLite.R")
> biocLite("DESeq")
gcc -std=gnu99 -I/usr/share/R/include -DNDEBUG      -fpic  -g -O2 -fstack-protector --param=ssp-buffer-size=4 -Wformat -Werror=format-security -D_FORTIFY_SOURCE=2 -g  -c pval.c -o pval.o
gcc -std=gnu99 -shared -L/usr/lib/R/lib -Wl,-Bsym

Bioconductor version 3.2 (BiocInstaller 1.20.3), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.2 (BiocInstaller 1.20.3), R 3.2.3 (2015-12-10).
Installing package(s) ‘DESeq’
trying URL 'https://bioconductor.org/packages/3.2/bioc/src/contrib/DESeq_1.22.1.tar.gz'
Content type 'application/x-gzip' length 1706087 bytes (1.6 MB)
downloaded 1.6 MB

* installing *source* package ‘DESeq’ ...
** libs
installing to /users/annashch/R/x86_64-pc-linux-gnu-library/3.2/DESeq/libs
** R
** inst
** preparing package for lazy loading
Note: the specification for S3 class “AsIs” in package ‘DBI’ seems equivalent to one from package ‘BiocGenerics’: not turning on duplicate class definitions for this class.
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (DESeq

## Running DESeq

We first clean up the column names in the counts.tab file located in **$WORK_DIR/results/signal**: 

In [10]:
%%bash 
echo $WORK_DIR
cd $WORK_DIR/results/signal
perl -i".bak" -pe '$_ = $.==1 ? do {$_ =~ s/\/[^\s]+\//samp/g; $_ =~ s/\"//g; $_ =~ s/\-/\./g; $_} : $_' counts.tab

bash: line 2: cd: /results/signal: No such file or directory
Can't open counts.tab: No such file or directory.


Now, we run DESeq with the following script: 

In [14]:
%%bash 
cd $WORK_DIR/results/signal 
Rscript $SRC_DIR/runDESeqTrainingCamp.r counts.tab DEoutput

Fatal error: cannot open file '/runDESeqTrainingCamp.r': No such file or directory


bash: line 1: cd: /results/signal: No such file or directory


This code will generate 2 files per time point pair. The first is the raw output from DESeq for all peaks. We will not have time to discuss everything in this file, but feel free to read the DESeq manual and see if you can understand it. The second, which ends in “sigPeakNames,” contains a list of the IDs of the differentially open peaks from ATAC‐seq. The p‐value cutoff for differential openness that we use is 0.05.